# Import

In [2]:
import configparser as cp
import os
import pandas as pd
import numpy as np
from glob import glob
import datetime
import re

# Data

In [3]:
config = cp.RawConfigParser()
config.read(r'config.txt')

# @slack_dir: directory of unziped raw slack data
# @proc_file: file where you want to store the combined&processed dataframe
slack_dir = config.get('main', 'slack_dir')
proc_file = config.get('main', 'proc_file')
slack_text_file = config.get('analysis', 'slack_text_file')

# Read essential channel and member info

In [12]:
# @chann_json: json file that stores channel metadata
# @user_json: json file that stores user metadata
chann_json = os.path.join('data', slack_dir, 'channels.json')
user_json  = os.path.join('data', slack_dir, 'users.json')

df_chann = pd.read_json(chann_json)
df_user = pd.read_json(user_json)

# @chann_lst: list of channel names
# @chann_member_dct: map of channels and corresponding member ids
chann_lst = df_chann['name'].tolist()
chann_member_dct = dict(zip(df_chann.name, df_chann.members))

# @user_lst: list of human user ids (excluding bot users or app users)
# @name_lst: list of human user first names
user_lst = df_user.loc[(df_user['is_bot'] == False) & (df_user['is_app_user'] == False)]['id'].tolist()
name_lst = df_user.loc[(df_user['is_bot'] == False) & (df_user['is_app_user'] == False)]['real_name'].tolist()

# Generate a valid first name that will be used to replace "@" symbols in texts
def get_first_name(name):
    name_split_lst = name.split(' ')
    first_name = name_split_lst[0]
    title_lst = ['Mr', 'Mrs', 'Miss', 'Ms', 'Mx', 'Sir', 'Dr', 'Cllr', 'Lady', 'Lord']
    for title in title_lst:
        if title in first_name and len(name_split_lst) > 0:
            first_name = name_split_lst[1]
            break
    return first_name

name_lst = [get_first_name(name) for name in name_lst]

# @user_name_dct: map of user ids and first names (for preprocessing)
user_name_dct = dict(zip(user_lst, name_lst))

# Combine all messages into a dataframe

In [13]:
# Combine a single channel
def merge_a_channel(channel):
    cur_dir = os.path.join('data', slack_dir, channel)
    file_lst = sorted(glob(cur_dir + '/*.json'))
    df_lst = []
    for file in file_lst:
        cur_df = pd.read_json(file)
        df_lst.append(cur_df)
    chann_df = pd.concat(df_lst, ignore_index=True, join='outer')
    chann_df.insert(0, 'channel', channel)
    return chann_df

# Combine all channels
def merge_all_channels(chann_lst):
    chann_df_lst = []
    for channel in chann_lst:
        chann_df = merge_a_channel(channel)
        chann_df_lst.append(chann_df)
    return pd.concat(chann_df_lst, ignore_index=True, join='outer')

# @df_raw: the combined raw dataframe
df_raw = merge_all_channels(chann_lst)

# Preprocessing: first step

### Extract only useful columns

In [116]:
# @attr_lst: list of useful metadata attributes
useful_attr_lst = [ 
    'type', 
    'subtype', 
    'ts', 
    'thread_ts',
    'channel',
    'user', 
    'text', 
    'reactions', 
    'replies',
    'parent_user_id']
df_processed = df_raw.filter(useful_attr_lst, axis=1)

# Replace NaN values with None
df_processed = df_processed.replace({np.nan: None})

### Extract only text messages

Ignore "XX has joined channel"

In [117]:
# Process 'type' and 'subtype'
df_processed = df_processed.loc[
    (df_processed['type']=='message') & 
    (df_processed['subtype'].isnull())]

useless_attr_list = ['type', 'subtype']
df_processed = df_processed.drop(columns=useless_attr_list)

### Transform UNIX timestamps to human-readable formats 

In [118]:
# Process 'ts'
def get_date(ts):
    return datetime.datetime.fromtimestamp(ts).date()

# Create 'date'
df_processed['date'] = df_processed['ts'].apply(get_date)

# Create 'week' number
min_date = df_processed['date'].min()

def get_week(date):
    delta = date - min_date
    days = delta.days
    return 1 + (days // 7)

df_processed['week'] = df_processed['date'].apply(get_week)

# Indentify interactions

A user connects to (interacts with) another user by replying to them or mentioning them

### Identify channel members

In [119]:
def get_members(channel):
    member_lst = chann_member_dct[channel]
    return member_lst

df_processed['members'] = df_processed['channel'].apply(get_members)

### Identify direct mentions

In [120]:
# Identify '<@user_id>' in 'text'
# Add 'mentions'
def get_mentions(text):
    regex = r'\<@(.*?)\>'
    mentions = re.findall(regex, text)
    mention_lst = list(set(mentions) & set(user_lst)) # must be a valid user 
    return mention_lst

df_processed['mentions'] = df_processed['text'].apply(get_mentions)

### Identify explicit addressees

In [121]:
# Explicit addressees = mentions + parent user
def get_explicit_addressees(row):
    if row['parent_user_id'] != None:
        explicit_addressees = [row['parent_user_id']] + row['mentions']
        explicit_addressees_set = set(explicit_addressees)
        speaker = row['user']
        if speaker in explicit_addressees_set:
            explicit_addressees_set.remove(row['user'])
        explicit_addressees_lst = list(explicit_addressees_set)
        return explicit_addressees_lst
    return row['mentions']

df_processed['explicit_addressees'] = df_processed.apply(lambda row: get_explicit_addressees(row), axis=1)

# Text cleaning

* Remove extra spaces and newlines
* Replace html escape charaters
* Replace URLs with a placeholder (\<url\>)
* Replace mentions (\<@UID\>) with valid user names
* Remove extremely short messages

In [122]:
# Remove extra spaces and newlines
def remove_spaces(text):
    text = ' '.join(text.split())
    return text

df_processed['text'] = df_processed['text'].apply(remove_spaces)

# Replace html escape charaters (unescaping)
unescape_dct = {'&lt;': '<',
                '&gt;': '>',
                '&quot;': '"',
                '&#39;':'\'',
                '&amp;': '&'}

def replace_escapes(text):
    for s in unescape_dct:
        c = unescape_dct[s]
        text = text.replace(s, c)
    return text

df_processed['text'] = df_processed['text'].apply(replace_escapes)

# Replace URLs with a placeholder '<url>'
def replace_urls(text):
    regex1 = r'https?://\S+'
    regex2 = r'http?://\S+'
    text = re.sub(regex1, 'url>', text)
    text = re.sub(regex2, 'url>', text)
    return text

df_processed['text'] = df_processed['text'].apply(replace_urls)

# Replace mentions ('<@user_id>') in 'text' with valid user names
def replace_mentions(text):
    regex = r'\<@(.*?)\>'
    mentions_found = re.findall(regex, text)
    mention_lst = list(set(mentions_found) & set(user_lst))
    for mention in mention_lst:
        real_name = user_name_dct[mention]
        text = text.replace('<@'+mention+'>', real_name)
    return text
        
df_processed['text'] = df_processed['text'].apply(replace_mentions)

#Remove extremely short texts
def remove_short(text):
    token_lst = text.split(' ')
    if len(token_lst) < 3:
        return ''
    return text
    
df_processed['text'] = df_processed['text'].apply(remove_short)

# Remove empty text messages
df_processed = df_processed.loc[df_processed['text'] != ''] 

# Last step

### Replace empty lists with None values

In [130]:
def replace_empty(what):
    if what == []:
        return None
    return what

df_processed['members'] = df_processed['members'].apply(replace_empty)
df_processed['mentions'] = df_processed['mentions'].apply(replace_empty)
df_processed['listeners'] = df_processed['listeners'].apply(replace_empty)

### Sort by date

In [123]:
df_processed = df_processed.sort_values(by=['date'])

### Rename column names

In [124]:
# df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
df_processed = df_processed.rename(columns={'user': 'speaker', 'explicit_addressees': 'listeners'})

### Reindex for LIWC

In [125]:
# Add one to index value & rename index column name as 'Row ID' (to match the convention of LIWC)
df_processed = df_processed.reset_index().drop(columns='index')
df_processed.index = df_processed.index + 1
df_processed.index.name = 'Row ID'

# Save

In [132]:
df_processed.to_csv(proc_file)

In [10]:
df_processed.to_csv(slack_text_file, columns=['text'], index=False)

In [ ]:
################################################################################
##################################### END ######################################
################################################################################